# **Walmart - Store Sales Forecasting**

A competição Walmart - Store Sales Forecasting tem como principal objetivo utilizar dados históricos de vendas de 45 lojas do Walmart localizadas em diferentes regiões para fazer predições de vendas por departamento, loja e semana observada.

Um ponto relevante neste estudo, são os eventos promocionais que a Walmart realiza antes dos feriados. Os principais feriados destacados na competição são: 

- Super Bowl: 12-Fev-10, 11-Fev-11, 10-Fev-12, 8-Fev-13
- Dia do Trabalho: 10-Set-10, 9-Set-11, 7-Set-12, 6-Set -13
- Ação de Graças: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13
- Natal: 31-Dez-10, 30-Dez-11, 28-Dez-12, 27-Dez -13


### Solução

A formulação do problema se caracteriza como um problema de série temporal. Como estamos tentando fazer predições de valores numéricos, como por exemplo: valor das vendas, preços, um volume, e assim por diante, temos uma modelagem preditiva do tipo regressão. Neste desafio nossa variável resposta é representada pela coluna Weekly_Sales.


Este notebook está dividido nas seguintes seções:

1.  **Leitura dos dados**
2.  **Análise Exploratória dos dados**
3.  **Engenharia de Features**
4.  **Buscar o melhor algoritmo**
5.  **Buscar os melhores parâmetros**
6.  **Testes**
7.  **Make Submission**
8.  **Próximos passos**

Cada seção inicialmente contem as funções que seram utilizadas, e em seguida uma breve explicação.


In [ ]:
from pandas.tseries.holiday import *

import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

!pip install plotly
!pip install shap
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import shap
shap.initjs()

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

from sklearn.model_selection import cross_val_predict, train_test_split, KFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor, GradientBoostingRegressor
!pip install xgboost
!pip install lightgbm
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor



# 1.  Leitura dos Dados

In [ ]:
def merge_data(df):

    df = df.merge(df_stores, 'left', 'Store')
    
    df = df.merge(df_features, 'left', ['Store', 'Date', 'IsHoliday'])
    
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

    return df

In [ ]:
# Paths

path_project = '../input/walmart-recruiting-store-sales-forecasting/'

path_train = path_project + 'train.csv.zip'

path_test = path_project + 'test.csv.zip'

path_stores = path_project + 'stores.csv'

path_features = path_project + 'features.csv.zip'

path_sample_submission = path_project + 'sampleSubmission.csv.zip'

In [ ]:
df_train = pd.read_csv(path_train)

df_test = pd.read_csv(path_test)

df_stores = pd.read_csv(path_stores)

df_features = pd.read_csv(path_features)

df_submission = pd.read_csv(path_sample_submission)

In [ ]:
df_train.head(1)

In [ ]:
df_train.describe()

In [ ]:
df_train = merge_data(df_train)

df_test = merge_data(df_test)

In [ ]:
df_train.head(1)

# 2. Análise Exploratória dos dados

In [ ]:
def get_percent_null(df):
    '''
    Esta função calcula a porcentagem de valores Null em dataframe
    '''
    return (
        
        (df.isnull().sum()/len(df) * 100).sort_values(ascending=False)
        
    ).to_frame(name='percent_null')

In [ ]:
def plot_heatmap(df, title=''):
    '''
    Esta função faz o plot heatmap do seanborn
    '''
    corr = df.corr()
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True
    with sns.axes_style("white"):
        f, ax = plt.subplots(figsize=(15, 15))
        ax.set_title(title)
        ax = sns.heatmap(corr, mask=mask, annot=True, vmax=.3, center=0, cmap=sns.color_palette("vlag", as_cmap=True))

In [ ]:
def plot_two_col(df, col_x='Type', col_y = 'count', type_plot='Bar', mode='markers'):

    '''
    Esta função faz o plot de duas variáveis, onde: 
    - col_x: Representa variável que assumiram o eixo x em cada um dos subplots
    - col_y: Representa a variável que assumira o eixo y nos dois subplots
    - type_plot: Tipo do plot, apenas são consideradas duas opções 'Bar' ou 'Scatter'
    - mode: Apenas utilizado quando type_plot é 'Scatter'
    '''
    
    data = None
    
    if type_plot == 'Bar': 
            
        if col_y is None: 
            col_y = 'count'

        df_ = df.groupby(by=[col_x]).size().reset_index(name='count').copy()  
        data = [go.Bar(x=df_[col_x], 
                       y=df_[col_y], 
                       marker=dict(color='#197278')
                )]

    elif type_plot == 'Scatter':

        if col_y is None: 
            col_y = 'Weekly_Sales'

        df = df_train.groupby(by=[col_x], as_index=False)[col_y].mean()
        data = [go.Scatter(x=df[col_x], 
                           y=df[col_y], 
                           marker=dict(color='#197278'), 
                           mode=mode)]
    else: 

        raise ValueError('Apenas duas opções são aceitas no parâmetro type_plot: Bar ou Sacetter.')
    
    fig = go.Figure(data=data)

    fig.update_layout(plot_bgcolor="white",showlegend= False, 
                      xaxis_title=col_x, yaxis_title=col_y, title=f'{col_x} vs {col_y}')
    fig.update_yaxes(showline=True, gridcolor='#dadae8')
    
    fig.show()

In [ ]:
def plot_scatter_per_year(df, col_x='week', col_y='Weekly_Sales', filter_month=None):

    '''
    Esta função faz o plot Scatter sobre duas variáveis, em cada ano, onde:
    - df: Dataframe
    - col_x: Coluna que ficará no eixo x
    - col_y: Coluna que ficará no eixo y
    - filter_month: Lista com o número dos meses que se deseja filtrar
    '''

    df = df[['Date', 'Weekly_Sales']].copy()
    df['week'] = df.Date.dt.isocalendar().week.astype('int64')
    df['year'] = df.Date.dt.year
    df['month'] = df.Date.dt.month

    fig = go.Figure()

    colors = ['#C17767', '#197278', '#512D38']

    title = f'{col_y} per mean {col_x}'

    if not filter_month is None:
        df = df[df['month'].isin(filter_month)]
        title += f' and only month: {filter_month}' 
    
    for year, color in zip(df['year'].unique().tolist(), colors):

        

        weekly_sales_year = df[df['year'] == year].groupby(col_x)[col_y].mean()

        fig.add_trace(
            
            go.Scatter(x=weekly_sales_year.index, y=weekly_sales_year.values,
                          mode='lines',
                          name= f'year {year}',
                          line=dict(color=color))
        )

    fig.update_layout(title=title, plot_bgcolor="white", yaxis_title=col_y, xaxis_title=col_x)

    fig.update_xaxes(showline=True, gridcolor='#dadae8')
    fig.update_yaxes(showline=True, gridcolor='#dadae8')

    fig.show()

### Investigando valores Nan
> df_train: Apenas as colunas de Markdown possuem valores Nan no dataframe de train

> df_test: Além das colunas que possuem prefixo markdown, as colunas Unemployment e CPI também possuem valores Nan


In [ ]:
get_percent_null(df_train)

In [ ]:
get_percent_null(df_test)

Logo, vamos precisar desenvolver alguma estratégia para lidar com os valores Nan, temos algumas opções:

1. Remover as colunas que possuem algum, ou todos os valores Nan
2. Utilizar alguma estratégia de imputação

Para as colunas que possuem o prefixo markdown, no dataframe de treino observamos que elas possuem mais da das linhas com valores Nan, utilizar estratégia de imputação de valores em colunas com baixíssimo preenchimento é muito arriscado, portanto, vamos excluir elas. Para as colunas Unemployment e CPI, vamos observar a correlação delas, para decidirmos se vale apena manter elas.

In [ ]:
plot_heatmap(df_train, 'Dataframe Train')

Analisando a matriz de correlação acima, observamos que as colunas Unemployment e CPI
possuem baixa correlação com nossa variável resposta Weekly_Sales. Observamos também que as colunas, Fuel_Price, Temperature apresentam baixa correlação com a variável resposta. As variáveis que possuem alguma correlação positiva com a variável resposta são: Dept, Size e IsHoliday. 

> Logo, por enquanto manteremos apenas as colunas: 
> 1. Store
> 2. Dept
> 3. Size
> 4. Type (categorical)
> 5. IsHoliday

Como descrito na Seção de métricas desse desafio¹, utilizaremos a métrica customizada WMAE, onde penalizamos mais os erros em feriados, visto que semanas com feriado possuem peso 5 vezes maior.

Com isso percebemos que as informações relacionadas aos feriados são muito importantes para o nosso modelo, vamos observar a representatividade dessa variável na nossa base de treino. Um ponto importante é que nem todos os feriados foram adicionados na base de treino, apenas os principais:

- Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13
- Labor Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13
- Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13
- Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13

Também vamos observar a representatividade e categorias que existem na coluna Type.

¹https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/overview/evaluation

In [ ]:
for col_x in ['Type', 'IsHoliday']:
    
    plot_two_col(df_train, col_x=col_x, col_y='count', type_plot='Bar')

Vamos observar também a relação entre as variáveis Dept e Size com nossa variável resposta.

In [ ]:
for col_x in ['Dept', 'Size']:
    
    plot_two_col(df_train, col_x=col_x, col_y='Weekly_Sales', type_plot='Scatter', mode='markers')

Com isso, concluímos que alguns depatamentos possuem faturamento muito superior aos outros, e que quanto maior a variável size também alcançamos um maior valor de vendas semanais.

Agora vamos observar como nossa variável resposta se comporta ao longo das semanas em cada ano.

In [ ]:
plot_scatter_per_year(df_train, col_x='week', col_y='Weekly_Sales')

## Adicionando feriados

> Como observado pelo participante [Avelino Caio](https://www.kaggle.com/avelinocaio/walmart-store-sales-forecasting), que notou que o pico das semanas 14 e 13 poderiam estar associadas ao feriado de páscoa (feriado não mapeado no nosso dataframe de treino), nos traz a reflexão de que outros feriado podem influenciar no aumentar das vendas também e que não estão mapeados no nosso dataframe de treino. 

A patir de uma busca rápida na internet [Public holidays in  the United States](https://en.wikipedia.org/wiki/Public_holidays_in_the_United_States), podemos encontrar outros feriados que poderiam influênciar também nas vendas dos departamentos. Um ponto de referência para decidirmos se um feriado pode ou não ser relevante para adicionar no nosso problema, seria analisando o impacto financeiro desse feriados no país. Em [https://nrf.com/insights/retail-holiday-and-seasonal-trends](https://nrf.com/insights/retail-holiday-and-seasonal-trends) encontramos a informação de vendas em bilhões de dolares americanos nos feriados. 

> É importante mencionar também que não foi fornecida as informações da localização das lojas e departamentos do nosso dataframe de treino, apenas sabemos que são lojas da Walmart, uma empresa american multinacional. Por sabermos que a sede da empresa se localiza nos EUA, estou tomando como base o calendário nacional de feriados de lá. 

Um outro ponto importante que nos ajuda a escolher esses feriados adicionais, é **observar os picos do gráfico anterior e observar se as semanas de pico estão relacionadas a algum feriado**. Seguindo essa linha encontramos um pico entre as semanas 26 e 27, mesma semana que ocorre o dia da independência americana (4 de julho). 

> Bom, com essas observações vamos incluir os feriados
> - **Páscoa** 
> - **Dia da independência Americana**

### Picos de vendas que não são feriados

> Um comportamento de senso comum observado aqui no Brasil, é o aumento do fluxo de pessoas em supermercados, lojas de conveniência, shoppings, entre outros, no **começo do mês devido ao pagamento do salário mensal das pessoas**. 

Isso me levou a pensar se esse comportamento poderia acontecer em outros paísese também. Utilizando como base os EUA, e uma rápida busca na internet [https://www.patriotsoftware.com/blog/payroll/pay-frequency-requirements-state-federal/](https://www.patriotsoftware.com/blog/payroll/pay-frequency-requirements-state-federal/), observamos que a freqência de pagamento dos funcionários é bem variada, mensal, quizenal, entre outros. 

Abaixo observamos o nosso gráfico anterior com um zoom nos meses Maio, Junho, Julho e Agosto, e vamos tentar observar algum padrão.

In [ ]:
plot_scatter_per_year(df_train, col_x='week', col_y='Weekly_Sales', filter_month=[5, 6, 7, 8])

As semans 22, 26, 27 e 31, que representam pico, são semanas do começo do mês. As semanas 30 e 20, representam o final do mês.

# 2. Engenharia de Features

In [ ]:
def get_df_holidays(df):
    '''
    Função que retorna um dataframe com as semanas de feriados do dataframe de treino
    + os novos feriados
    '''
    holiday_list = [
          '2010-02-12', '2011-02-11', '2012-02-10', '2013-02-08',
          '2010-09-10','2011-09-09', '2012-09-07', '2013-09-06',
          '2010-11-26', '2011-11-25', '2012-11-23', '2013-11-29',
          '2010-12-31', '2011-12-30', '2012-12-28', '2013-12-27'
    ]

    df_holiday = pd.DataFrame(zip(holiday_list, [True] * len(holiday_list)), 
                              columns=['day', 'is_holiday'])

    df_holiday['day'] = pd.to_datetime(df_holiday['day'], format='%Y-%m-%d')

    df['year'] = df.Date.dt.year
    df['week'] = df.Date.dt.isocalendar().week.astype('int64')

    list_new_holidays = [
                      
                    # Páscoa 
                    (2010, 13), (2011, 16), (2012, 14), (2013, 13),

                    # Dia da Independência Americana (04/07)
                    (2010, 26), (2011, 26), (2012, 27), (2013, 27),
    ] 

    for year, week in list_new_holidays:
        df.loc[((df.year == year) & (df.week == week)), 'is_holiday'] = True

    df = df[df['is_holiday'] == True][['Date', 'is_holiday']]

    df.columns = ['day', 'is_holiday']

    return pd.concat([df_holiday, df], ignore_index=True).drop_duplicates().reset_index(drop=True) 

In [ ]:
def count_holidays_per_key(df, keys, col_name):
    '''
    Função que calcula a quantidade de feriados em relação a alguma outra feature.
    Ex: Quantidade de feriados por mês, ou por semana
    '''
    df_grouped =  df.groupby(keys, as_index=False)['is_holiday'].count()

    df_grouped.columns = keys + [col_name]
    
    return (
    
        df.merge(

                df_grouped, how='left', on=keys

            )
    
    ).fillna(0)

In [ ]:
def create_features_holidays(df, df_holidays, start, end):

    '''
    Função que cria features booleanas que informam se a semana é do 
    - início do mês 
    - quizena do mês
    - final do mês
    '''
        
    all_days = pd.date_range(start, end, freq='D').to_series()
    
    df_all_days = pd.DataFrame(all_days, columns=['day'])

    df_all_days.loc[
        
            df_all_days.day.dt.strftime('%d') == '15', 'is_fortnight'

    ] = True

    df_all_days['is_fortnight'] = df_all_days['is_fortnight'].fillna(value=False)
    
    df_all_days['year'] = df_all_days.day.dt.year
    
    df_all_days['week'] = df_all_days.day.dt.isocalendar().week.astype('int64')
    
    df_all_days['week_day'] = df_all_days.day.dt.dayofweek
    
    df_all_days['is_month_start'] = df_all_days.day.dt.is_month_start
    
    df_all_days['is_month_end'] = df_all_days.day.dt.is_month_end

    df_all_days = df_all_days.merge(df_holidays, 'left', 'day')
    
    df_all_days['is_holiday'] = df_all_days['is_holiday'].fillna(value=False)
    
    df_week_year = df_all_days.groupby(['week', 'year'], as_index=False)

    df_week_year = df_week_year.any()
    
    df_week_year = df_week_year[[
    
        'week', 'year', 'is_month_start', 'is_fortnight', 
        'is_month_end', 'is_holiday'
    ]]
    
    df_week_year.columns = [
    
        'week', 'year', 'is_week_start_month', 'is_week_fortnight', 
        'is_week_end_month', 'is_holiday'
    ]

    return df.merge(
        
        df_week_year, 'left', ['week', 'year']

    ).fillna(value=False)    

In [ ]:
def create_features(df):

    '''
    Função que criar as features adicionadas ao dataframe de treinamento:
    - year
    - month
    - week
    - 'is_week_start_month'
    - 'is_week_fortnight' 
    - 'is_week_end_month'
    - 'is_holiday'
    - 'qt_holiday_week'
    '''
       
    start = pd.Timestamp(df.Date.dt.date.min()).to_pydatetime()
    
    end = pd.Timestamp(df.Date.dt.date.max()).to_pydatetime()
    
    df_holidays = get_df_holidays(df.copy())

    df['year'] = df.Date.dt.year
    
    df['week'] = df.Date.dt.isocalendar().week.astype('int64')
    
    df['month'] = df.Date.dt.month
    
    df = create_features_holidays(df.copy(), df_holidays, start, end)

    df = count_holidays_per_key(df.copy(), ['year', 'week'], 'qt_holiday_week')

    return df.drop_duplicates().reset_index(drop=True)

In [ ]:
def convert_to_numeric(df):
    '''
    Função que converte todas as colunas pra float e preenche valores nan.
    '''

    list_col_numeric = [col for col in df if df[col].dtype != 'O']

    list_col_cat = [col for col in df if df[col].dtype == 'O']

    for col in df.columns:

        if col in list_col_numeric:
            
            df[col] = df[col].fillna(-9999).astype('float64')
            
        elif col in list_col_cat:
            
            df[col] = df[col].fillna('ND')

    return df

In [ ]:
def prepare_dataframe(df, is_test = False):

    '''
    Função que chama a criação de feature, seleciona as features usadas no treinamento
    '''
    
    df = create_features(df)
    
    col_selected = [
    
           'Store', 'Dept', 'Type', 'Size', 'is_holiday',
           'year', 'week', 'is_week_start_month', 'month',
           'is_week_fortnight', 'is_week_end_month',
           'qt_holiday_week', 'Weekly_Sales'
    ]
    type_df = 'df_train'

    if is_test:
        
        col_selected.remove('Weekly_Sales')
        type_df = 'df_test'

    plot_heatmap(df, f'{type_df} After add all the features')

    df = df[col_selected]
    
    df = convert_to_numeric(df) 

    plot_heatmap(df, f'{type_df} After removing some features')

    return df

In [ ]:
def get_make_col_transformer(df):

    '''
    Função que cria um pipeline, onde definimos a estratégia de transformação das colunas 
    categóricas e a forma de normalização que será utilizada
    '''

    list_col_numeric = [col for col in df if df[col].dtype != 'O']

    list_col_cat = [col for col in df if df[col].dtype == 'O']

    if 'Weekly_Sales' in list_col_numeric: list_col_numeric.remove('Weekly_Sales')

    list_categories = [df[column].unique() for column in df[list_col_cat]]

    encoder_col_cat = make_pipeline(
        OrdinalEncoder(categories=list_categories)
    )

    normalize_col_numerics = make_pipeline(
        StandardScaler()
    )
    
    return make_column_transformer(
        
        (encoder_col_cat, list_col_cat),
        (normalize_col_numerics, list_col_numeric)
        
    )

### Features Criadas

- year
- month
- week
- is_week_start_month
- is_week_fortnight
- is_week_end_month
- is_holiday
- qt_holiday_week

In [ ]:
df_train  = prepare_dataframe(df_train)

df_test = prepare_dataframe(df_test, is_test=True)

transform_columns = get_make_col_transformer(df_train)

# 4. Buscando o melhor algoritmo

Agora que definimos nossas features, precisamos encontrar o melhor algoritmo para fazermos a regressão.

In [ ]:
X = df_train.drop(['Weekly_Sales'], axis=1)
y = df_train['Weekly_Sales']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
def WMAE(dataset, real, predicted, col_target='is_holiday'):
    """
    Métrica da competição
    """
    weights = dataset[col_target].apply(lambda x: 5 if x else 1)
    return np.round(np.sum(weights*abs(real-predicted))/(np.sum(weights)), 2)

In [ ]:
models = {

    'LinearRegression': LinearRegression(),
    
    'XGBRegressor': XGBRegressor(),
    
    'RandomForestRegressor': RandomForestRegressor(verbose = False, n_jobs = -1),
       
    'LGBMRegressor': LGBMRegressor(),
    
    'ExtraTreesRegressor' : ExtraTreesRegressor(verbose = False, n_jobs = -1),
    
    'BaggingRegressor' : BaggingRegressor(verbose = False, n_jobs = -1),
    
    'GradientBoostingRegressor': GradientBoostingRegressor()
}

best_nome_model = None
best_model = None
best_error = math.inf
list_metrics = []

In [ ]:
for name in models:
  
    print(f'\n\n\n{name}')
    regressor = models[name]
                       
    training_pipeline = make_pipeline(
        transform_columns,
        regressor
    )

    kfold = KFold(n_splits=3)
    
    y_pred = cross_val_predict(
        training_pipeline,
        X_train,
        y_train,
        cv = kfold,
    )
    
    wmae_metric = WMAE(X_train, y_train, y_pred)
    
    print(f'\nModel {name} Metrica WMAE', wmae_metric)
    
    list_metrics.append((name, wmae_metric))
    
    if wmae_metric < best_error:
        best_error = wmae_metric
        best_model = regressor
        best_nome_model = name

In [ ]:
list_metrics

In [ ]:
best_model

In [ ]:
best_error

Buscando os melhores parâmetros do melhor modelo 

In [ ]:
def find_best_paramns(best_model, df_train, sample_train=0.5, param_grid=None, 
                      type_search='randomized', n_iter=None):
    
    '''
    Função que encontra os melhores parâmetros para o melhor modelo
    
    best_model: Instância do melhor modelo encontrado
    df_train: Dataframe de treinamento
    sample_train: Sample utilizada para fazer a otimização de parâmetros, se 1 considera todo o dataframe
    param_grid: Dict com os parâmetros e valores a serem testados
    type_search: Tipo da busca otimizada, pode ser: 'grid' ou 'randomized' 
    '''
    
    df_sample = df_train.sample(frac=sample_train)
    
    X, y = df_sample.drop(['Weekly_Sales'], axis=1), df_sample['Weekly_Sales']

    X_train, _, y_train, _ = train_test_split(X, y, test_size=0.3)

    sample_train_pipeline = Pipeline([
        ("transform_columns", transform_columns),
        ("model", best_model)
    ])

    kfold = KFold(n_splits=3)
    
    if param_grid is None:
      
      param_grid = {
          
          #'model__bootstrap': [True],
          #'model__max_features': ['auto', 'log2'],
          #'model__min_samples_split': [2, 3],
          'model__max_depth': [None, 30],
          'model__n_estimators': [60, 150, 200]

      }

    if type_search == 'randomized':
        
        model_search = RandomizedSearchCV(
            sample_train_pipeline,
            param_distributions = param_grid,
            cv = kfold,
            n_iter=n_iter,
            n_jobs = -1
        )
        
    elif type_search == 'grid':
        
        model_search = GridSearchCV(
            sample_train_pipeline,
            param_grid = param_grid,
            cv = kfold,
            n_jobs = -1
        )
    
    else:
        
        raise ValueError('Apenas duas opções são aceitas no parâmetro type_search: randomized ou grid.')

    return model_search.fit(X_train, y_train).best_params_

In [ ]:
best_params = find_best_paramns(best_model, df_train, sample_train=1, type_search='grid')

In [ ]:
best_params

In [ ]:
paramns_name = []

for key, value in best_params.items():
    paramns_name.append(key.replace('model__', ''))

best_params_ = dict(zip(paramns_name, best_params.values()))

In [ ]:
best_params_

Treinando com os melhores parâmetros

In [ ]:
def fit_best_model(best_model, X_train, y_train, X_test, y_test):

    model = best_model.set_params(**best_params_)

    training_pipeline = Pipeline([
              ("transform_columns", transform_columns),
              ("model", model)
    ])

    training_pipeline.fit(X_train, y_train)

    y_pred = training_pipeline.predict(X_test)

    wmae_metric = WMAE(X_test, y_test, y_pred)

    print(f'\nBest Model WMAE: ', wmae_metric)

    return training_pipeline, wmae_metric

In [ ]:
best_model_, wmae_metric = fit_best_model(best_model, X_train, y_train, X_test, y_test)

In [ ]:
def plot_feature_importance(feature_importance, columns):
    '''
    Função que plota o feature importance do melhor modelo
    '''
    df_importance = pd.DataFrame(
    
        zip(columns, feature_importance), 
        columns=['Feature', 'Importance']

    ).sort_values(by=['Importance'])

    fig = go.Figure(data=[
        go.Bar(y=df_importance['Feature'], 
               x=df_importance['Importance'], 
               marker=dict(color='#197278'),
               orientation='h'
              )
    ])

    fig.update_layout(plot_bgcolor="white",showlegend= False, title='Feature Importance', 
                      xaxis_title='Importance', yaxis_title='Feature')
    fig.update_yaxes(showline=True, gridcolor='#dadae8')

    fig.show()

In [ ]:
plot_feature_importance(best_model_.steps[1][1].feature_importances_, X_train.columns)

Das variáveis que foram criadas e tiveram algum impacto significativo, foram apenas:

- week
- qt_holiday_week
- month
- is_week_end_month
- year


# 6. Testes

In [ ]:
y_pred = best_model_.predict(df_test)

df_test['Weekly_Sales'] = y_pred

# 7. Submission

In [ ]:
df_submission['Weekly_Sales'] = y_pred

df_submission.to_csv('submission.csv',index=False)

In [ ]:
df_submission

# 8. Próximos passos

1. Aumentar o número de parâmetros na otimização de parâmetros
2. Refinar o processo de engenharia de features, tentar encontrar mais features que expliquem as variações de vendas entre as semanas
3. Tentar utilizar outros algoritmos
